In [1]:
cd '/userdata/pmavrodi/Projekte/Stao/src'

/userdata/pmavrodi/Projekte/Stao/src


In [2]:


import logging
import pandas as pd
import numpy as np
import configparser

from simple_logging.custom_logging import setup_custom_logger
from input_reader.input_reader import get_input

settingsFile = "settings.cfg"

# logger einrichten
LOGGING_LEVEL = logging.INFO
logger = setup_custom_logger('GM_LOGGER', LOGGING_LEVEL, flog="logs/gm.log")

In [3]:
pd.__version__

'0.18.1'

# Testing core algo

In [30]:
stores_migros_pd.head()

,ID,RETAILER,FORMAT,STRASSE,PLZ,ORT,VERKAUFSFLAECHE_TOTAL,VERKAUFSFLAECHE,Y,X,RELEVANZ,PROFIT_KST,UMSATZ_FOOD,UMSATZ_FRISCHE,UMSATZ_NEAR_NONFOOD,UMSATZ_FACHMARKT,own_hektar_id
OBJECTID,,,,,,,,,,,,,,,,,
6,SM_MIG_61607_15939,Migros,M,Kanderstegstrasse 31,3714,Frutigen,878.621,519.42,616073.9153,159391.5490,1.0,33370.0,2714265.89,5007897.01,1647566.21,149192.23,61601593
8,SM_MIG_66835_23607,Migros,MM,Kirchfeldstrasse 8,5630,Muri AG,1787.864,1001.19,668350.1017,236074.5430,1.0,33934.0,4973971.80,10746847.03,3218149.87,476688.61,66832360
9,SM_MIG_55810_14347,Migros,M,Rue du Léman 9,1815,Clarens,1026.810,622.98,558105.4795,143471.3736,1.0,63991.0,4432271.32,10699936.16,2424134.69,233238.81,55811434
11,SM_MIG_49525_11909,Migros,M,Route de Peney 2-4,1214,Vernier,1257.324,649.41,495252.6938,119091.9983,1.0,44860.0,3537224.77,7729667.80,1861354.71,233174.78,49521190
13,SM_MIG_63399_12693,Migros,MM,Brückenweg 10,3930,Visp,2230.170,1101.84,633999.7599,126933.4256,1.0,134480.0,6229558.90,11250268.13,5737413.46,781103.84,63391269


In [4]:
(stores_pd, stores_migros_pd, drivetimes_pd, haushalt_pd) = get_input(settingsFile, logger)
relevant_hektars = set(drivetimes_pd.loc[stores_migros_pd.ID]['hektar_id'])
drivetimes_rel_hektars_pd = drivetimes_pd[drivetimes_pd['hektar_id'].isin(relevant_hektars)]

In [29]:
drivetimes_pd.head()

,fahrzeit,hektar_id
filiale_id,,
SM_MIG_61607_15939,21,61341718
SM_MIG_61607_15939,21,61341719
SM_MIG_61607_15939,21,61341722
SM_MIG_61607_15939,22,61341730
SM_MIG_61607_15939,23,61341731


In [6]:
stores_pd.head()


,ID,RETAILER,FORMAT,STRASSE,PLZ,ORT,VERKAUFSFLAECHE_TOTAL,VERKAUFSFLAECHE,Y,X,RELEVANZ,PROFIT_KST,UMSATZ_FOOD,UMSATZ_FRISCHE,UMSATZ_NEAR_NONFOOD,UMSATZ_FACHMARKT,own_hektar_id
OBJECTID,,,,,,,,,,,,,,,,,
6,SM_MIG_61607_15939,Migros,M,Kanderstegstrasse 31,3714,Frutigen,878.621,519.420,616073.9153,159391.5490,1.0,33370.0,2714265.89,5007897.01,1647566.21,149192.23,61601593
8,SM_MIG_66835_23607,Migros,MM,Kirchfeldstrasse 8,5630,Muri AG,1787.864,1001.190,668350.1017,236074.5430,1.0,33934.0,4973971.80,10746847.03,3218149.87,476688.61,66832360
9,SM_MIG_55810_14347,Migros,M,Rue du Léman 9,1815,Clarens,1026.810,622.980,558105.4795,143471.3736,1.0,63991.0,4432271.32,10699936.16,2424134.69,233238.81,55811434
10,SM_MIG_49997_11718,Migros,SPEZ,Rue du Conseil-Général 20,1205,Genève,157.476,157.476,499972.6443,117183.8769,1.0,NaN,NaN,NaN,NaN,NaN,49991171
11,SM_MIG_49525_11909,Migros,M,Route de Peney 2-4,1214,Vernier,1257.324,649.410,495252.6938,119091.9983,1.0,44860.0,3537224.77,7729667.80,1861354.71,233174.78,49521190


In [5]:
drivetimes_rel_hektars_pd.head()

,fahrzeit,hektar_id
filiale_id,,
SM_MIG_61607_15939,21,61341718
SM_MIG_61607_15939,21,61341719
SM_MIG_61607_15939,21,61341722
SM_MIG_61607_15939,22,61341730
SM_MIG_61607_15939,23,61341731


In [7]:
drivetimes_rel_hektars_stores_pd = drivetimes_rel_hektars_pd.merge(
        stores_pd[['ID','FORMAT', 'VERKAUFSFLAECHE', 'VERKAUFSFLAECHE_TOTAL', 'RELEVANZ']], left_index=True,
    right_on='ID', how='inner')

In [10]:
len(drivetimes_rel_hektars_pd)

110246169

In [11]:
len(drivetimes_rel_hektars_stores_pd)

109974613

In [12]:
drivetimes_rel_hektars_stores_pd.head()

,fahrzeit,hektar_id,ID,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ
OBJECTID,,,,,,,
6,21,61341718,SM_MIG_61607_15939,M,519.42,878.621,1.0
6,21,61341719,SM_MIG_61607_15939,M,519.42,878.621,1.0
6,21,61341722,SM_MIG_61607_15939,M,519.42,878.621,1.0
6,22,61341730,SM_MIG_61607_15939,M,519.42,878.621,1.0
6,23,61341731,SM_MIG_61607_15939,M,519.42,878.621,1.0


In [13]:
enriched_pd = drivetimes_rel_hektars_stores_pd.merge(haushalt_pd[['H14PTOT']], left_on='hektar_id', right_index=True)

In [14]:
enriched_pd['H14PTOT_corrected'] = enriched_pd['H14PTOT'].fillna(1)

In [16]:
enriched_pd.head()

,fahrzeit,hektar_id,ID,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ,H14PTOT,H14PTOT_corrected
OBJECTID,,,,,,,,,
6,21,61341718,SM_MIG_61607_15939,M,519.42,878.621,1.0,1,1
20,28,61341718,SM_MIG_60500_19797,M,755.95,1300.694,1.0,1,1
530,12,61341718,SM_MIG_61274_17700,FM,820.12,1330.134,1.0,1,1
836,25,61341718,SM_MIG_60926_19134,MM,850.50,1879.651,1.0,1,1
949,29,61341718,SM_MIG_60277_19842,M,334.03,494.532,1.0,1,1


In [18]:
migros_only_pd = enriched_pd[enriched_pd['FORMAT'].isin(list(set(stores_migros_pd['FORMAT'])))]

In [23]:
migros_only_pd.groupby('ID').agg( lambda x: len(x))

,fahrzeit,hektar_id,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ,H14PTOT,H14PTOT_corrected
ID,,,,,,,,
SM_MIG_48928_12169,8858,8858,8858,8858.0,8858.0,8858.0,8858,8858
SM_MIG_49525_11909,11407,11407,11407,11407.0,11407.0,11407.0,11407,11407
SM_MIG_49539_12095,11333,11333,11333,11333.0,11333.0,11333.0,11333,11333
SM_MIG_49640_11777,11323,11323,11323,11323.0,11323.0,11323.0,11323,11323
SM_MIG_49743_11609,10982,10982,10982,10982.0,10982.0,10982.0,10982,10982
SM_MIG_49752_12087,12168,12168,12168,12168.0,12168.0,12168.0,12168,12168
SM_MIG_49768_11948,11684,11684,11684,11684.0,11684.0,11684.0,11684,11684
SM_MIG_49781_11376,10585,10585,10585,10585.0,10585.0,10585.0,10585,10585
SM_MIG_49823_12053,11698,11698,11698,11698.0,11698.0,11698.0,11698,11698


In [ ]:
drivetimes_pd = pd.read_csv(drivetimes, sep=',', header=None, names=['filiale_id', 'fahrzeit', 'hektar_id'],
                                    index_col=0, nrows=110299436)

In [ ]:
stores_pd[stores_pd['FORMAT'].isin(['M', 'MM', 'MMM', 'FM'])]

# Einige Daten issues

In [ ]:
# get the input
(stores_pd, stores_migros_pd, drivetimes_pd, drivetimes_migros_pd, haushalt_pd) = get_input(settingsFile, logger)

In [ ]:
drivetimes_pd.head(3)

In [ ]:
print('Number of entries in drivetimes %d: ' % len(drivetimes_pd))

# Duplikate in drivetimes

In [ ]:
drivetimes_pd2 = drivetimes_pd.reset_index()
drivetimes_pd2 = drivetimes_pd2.set_index(['filiale_id', 'fahrzeit', 'hektar_id'])

In [ ]:
drivetimes_pd2.head(3)

In [ ]:
drivetimes_duplicates = drivetimes_pd2[ drivetimes_pd2.index.duplicated()]

In [ ]:
print('Number of duplicates: %d' % len(drivetimes_duplicates))

In [ ]:
drivetimes_duplicates.head(10)

In [ ]:
a=drivetimes_pd.loc['SM_MIG_68294_26394']
a[ (a['fahrzeit']==11) & (a['hektar_id']==67672614) ]

In [ ]:
drivetimes_duplicates.to_csv('/userdata/pmavrodi/Projekte/Stao/output/drivetimes_duplicates.csv')

# Duplikate in drivetimes_migros_pd

In [ ]:
drivetimes_pd3 = drivetimes_migros_pd.reset_index()
drivetimes_pd3 = drivetimes_pd3.set_index(['filiale_id', 'fahrzeit', 'hektar_id'])

In [ ]:
drivetimes_pd3.head(4)

In [ ]:
drivetimes_duplicates2 = drivetimes_pd3[ drivetimes_pd3.index.duplicated()]

In [ ]:
print('Number of duplicates: %d' % len(drivetimes_duplicates2))

In [ ]:
drivetimes_duplicates2.head(3)

# Filiale in drivetimes, aber nicht in stores_sm. Betreffen sind LAT, RLAT, usw

In [ ]:
# get all relevant hektars, i.e. those from which a Migros store is reachable
relevant_hektars = set(drivetimes_migros_pd['hektar_id'])

In [ ]:
drivetimes_rel_hektars_pd = drivetimes_pd[drivetimes_pd['hektar_id'].isin(relevant_hektars)]

In [ ]:
drivetimes_rel_hektars_stores_pd = drivetimes_rel_hektars_pd.join(
        stores_pd[['FORMAT', 'VERKAUFSFLAECHE', 'VERKAUFSFLAECHE_TOTAL', 'RELEVANZ']], how='left')
len(drivetimes_rel_hektars_pd)

In [ ]:
b=drivetimes_rel_hektars_stores_pd[pd.isnull(drivetimes_rel_hektars_stores_pd['VERKAUFSFLAECHE_TOTAL']) &
                                            pd.isnull(drivetimes_rel_hektars_stores_pd['RELEVANZ'])&
                                            pd.isnull(drivetimes_rel_hektars_stores_pd['FORMAT'])].index.unique()

In [ ]:
b

In [ ]:
drivetimes_rel_hektars_stores_pd[~pd.isnull(drivetimes_rel_hektars_stores_pd['VERKAUFSFLAECHE_TOTAL'])].head(2)

In [ ]:
np.savetxt('/userdata/pmavrodi/Projekte/Stao/output/filiale_nur_drivetimes.txt', b,  fmt='%s')

# Hektare von drivetimes_sm ohne HH. Betrifft ist (lokal) Umsatzpotenztal

In [ ]:
enriched_pd = drivetimes_rel_hektars_stores_pd[~pd.isnull(drivetimes_rel_hektars_stores_pd['VERKAUFSFLAECHE_TOTAL'])].join(haushalt_pd['H14PTOT'], on='hektar_id')

In [ ]:
enriched_pd.head(5)

In [ ]:
hektars_ohne_HH_info = enriched_pd[pd.isnull(enriched_pd['H14PTOT'])]['hektar_id'].unique()

In [ ]:
len(hektars_ohne_HH_info)

In [ ]:
np.savetxt('/userdata/pmavrodi/Projekte/Stao/output/hektaren_ohne_HH_info.txt', hektars_ohne_HH_info, fmt='%s')

# DEBUGGING MIT BOJAN

In [31]:
import pickle

In [32]:
enriched = pd.read_pickle('/userdata/pmavrodi/Projekte/Stao/output/enriched_pd_with_duplicates.pkl')

In [40]:
enriched[enriched['ID'] == 'SM_MIG_73526_18182'].head(10)


,OBJECTID,fahrzeit,hektar_id,ID,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ,H14PTOT,H14PTOT_corrected,LAT,LAT2,RLAT,RLAT2,Marktanteil,Marktanteil2,LokalUP,LokalUP2,LokalUP_corrected,LokalUP2_corrected
89152178,583,9,73891840,SM_MIG_73526_18182,MM,553.54,1480.807,1.0,8,8,17769.684,6642.48,3712.613203,1387.810776,0.164257,0.029534,10249.619842,1842.902383,10249.619842,1842.902383
89152264,583,10,73891841,SM_MIG_73526_18182,MM,553.54,1480.807,1.0,9,9,17769.684,6642.48,2510.034581,938.275239,0.115616,0.015670,8116.229232,1100.051091,8116.229232,1100.051091
89152351,583,10,73891842,SM_MIG_73526_18182,MM,553.54,1480.807,1.0,5,5,17769.684,6642.48,2510.034581,938.275239,0.115548,0.015663,4506.373391,610.865804,4506.373391,610.865804
89152438,583,10,73891843,SM_MIG_73526_18182,MM,553.54,1480.807,1.0,2,2,17769.684,6642.48,2510.034581,938.275239,0.116274,0.015699,1813.869236,244.898031,1813.869236,244.898031
89152525,583,9,73891851,SM_MIG_73526_18182,MM,553.54,1480.807,1.0,2,2,17769.684,6642.48,3712.613203,1387.810776,0.127585,0.015751,1990.321733,245.714496,1990.321733,245.714496
89152618,583,9,73891852,SM_MIG_73526_18182,MM,553.54,1480.807,1.0,5,5,17769.684,6642.48,3712.613203,1387.810776,0.127576,0.015751,4975.446280,614.276763,4975.446280,614.276763
89152711,583,9,73891853,SM_MIG_73526_18182,MM,553.54,1480.807,1.0,2,2,17769.684,6642.48,3712.613203,1387.810776,0.127030,0.015729,1981.673581,245.372494,1981.673581,245.372494
89152805,583,9,73891854,SM_MIG_73526_18182,MM,553.54,1480.807,1.0,4,4,17769.684,6642.48,3712.613203,1387.810776,0.114931,0.014855,3585.857361,463.468302,3585.857361,463.468302
89152899,583,9,73891855,SM_MIG_73526_18182,MM,553.54,1480.807,1.0,7,7,17769.684,6642.48,3712.613203,1387.810776,0.114733,0.014846,6264.434723,810.600537,6264.434723,810.600537
89152993,583,9,73891856,SM_MIG_73526_18182,MM,553.54,1480.807,1.0,7,7,17769.684,6642.48,3712.613203,1387.810776,0.117513,0.014969,6416.222042,817.294025,6416.222042,817.294025


In [41]:
20*(1480.807-1000)/1500 + 60

66.41076

In [39]:
0.374*60

22.44

In [ ]:
66.41076*10